Data Warehouse, SQL Query and KPI
* Nama: Muhammad Rifat Bagus Adikusuma
* NIM: 20/456371/TK/50501
* Prodi : Teknologi Informasi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sqlite3

In [4]:
import pandas as pd

In [5]:
assesstments= pd.read_csv("/content/drive/MyDrive/Dataset BigData/assessments.csv")
courses =pd.read_csv("/content/drive/MyDrive/Dataset BigData/courses.csv")
stundentAssessment =pd.read_csv("/content/drive/MyDrive/Dataset BigData/studentAssessment.csv")
studentInfo =pd.read_csv("/content/drive/MyDrive/Dataset BigData/studentInfo.csv")
studentRegistration =pd.read_csv("/content/drive/MyDrive/Dataset BigData/studentRegistration.csv")
studentVle =pd.read_csv("/content/drive/MyDrive/Dataset BigData/studentVle.csv")
vle =pd.read_csv("/content/drive/MyDrive/Dataset BigData/vle.csv")

In [6]:
value_counts = studentInfo['final_result'].value_counts()

# Print the count of every unique value
print("Count of every unique value in 'course_code':")
for value, count in value_counts.items():
     print(f"{value}: {count}")

Count of every unique value in 'course_code':
Pass: 12361
Withdrawn: 10156
Fail: 7052
Distinction: 3024


In [7]:
conn = sqlite3.connect(':memory:')

In [8]:
import sqlite3

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Execute the SQL query to create tables and insert data
assesstments.to_sql('assesstments', conn, index=False)
courses.to_sql('courses', conn, index=False)
stundentAssessment.to_sql('studentAssesstment', conn, index=False)
studentInfo.to_sql('studentInfo', conn, index=False)
studentRegistration.to_sql('studentRegistration', conn, index=False)
studentVle.to_sql('studentVle', conn, index=False)
vle.to_sql('vle', conn, index=False)

# Commit the changes
conn.commit()

# Execute the SQL query to fetch table names
show_tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(show_tables_query)

# Fetch and print the table names
tables = cursor.fetchall()
print("Existing tables:")
for table in tables:
    print(table[0])

# Close the connection
# conn.close()


Existing tables:
assesstments
courses
studentAssesstment
studentInfo
studentRegistration
studentVle
vle


In [9]:
cursor = conn.cursor()
query = '''
SELECT
    code_module,
    (COUNT(DISTINCT CASE WHEN final_result IN ('Pass', 'Distinction') THEN id_student END) * 100.0 / COUNT(DISTINCT id_student)) AS completion_rate
FROM
    studentInfo
GROUP BY
    code_module
ORDER BY
  completion_rate DESC;
'''
result = pd.read_sql_query(query, conn)
print(result)

  code_module  completion_rate
0         AAA        74.578652
1         GGG        59.960396
2         EEE        57.712487
3         FFF        49.317291
4         BBB        48.803952
5         DDD        44.630643
6         CCC        39.473065


In [10]:
query = '''
SELECT
    code_module,
    (COUNT(DISTINCT CASE WHEN final_result = 'Distinction' THEN id_student END) * 100.0 / COUNT(DISTINCT id_student)) AS distinct_completion_rate
FROM
    studentInfo
GROUP BY
    code_module
ORDER BY
  distinct_completion_rate DESC;

'''
result_graduate_distinction = pd.read_sql_query(query, conn)
print(result_graduate_distinction)

  code_module  distinct_completion_rate
0         GGG                 15.683168
1         EEE                 12.451906
2         CCC                 11.714891
3         FFF                  9.057726
4         BBB                  8.801352
5         DDD                  6.549248
6         AAA                  6.179775


In [11]:
query = '''
SELECT
    a.code_module,
    AVG(sa.score) AS average_score
FROM
    studentAssesstment sa
JOIN
    assesstments a ON sa.id_assessment = a.id_assessment
GROUP BY
    a.code_module
ORDER BY
    average_score DESC;

'''
result_average_assesstment_score = pd.read_sql_query(query, conn)
print(result_average_assesstment_score)

  code_module  average_score
0         EEE      81.180066
1         GGG      79.700493
2         FFF      77.707590
3         BBB      76.706368
4         CCC      73.261398
5         DDD      70.090800
6         AAA      69.030515


In [12]:
query = '''
SELECT
    code_module,
    AVG(DISTINCT date) AS average_time_spent,
    COUNT(DISTINCT id_student) AS unique_students
FROM
    studentVle
GROUP BY
    code_module
ORDER BY
    average_time_spent DESC;


'''
result_engagement = pd.read_sql_query(query, conn)
print(result_engagement)

  code_module  average_time_spent  unique_students
0         BBB          128.457143             6484
1         GGG          126.500000             2359
2         FFF          125.500000             6799
3         EEE          125.500000             2634
4         CCC          125.500000             3852
5         AAA          122.500000              702
6         DDD          118.500000             5407


In [13]:
query = '''
SELECT
    region,
    COUNT(*) AS total_students,
    (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM studentInfo)) AS percentage_of_students
FROM
    studentInfo
GROUP BY
    region
ORDER BY
    percentage_of_students DESC;
'''
result_studentbyregion = pd.read_sql_query(query, conn)
print(result_studentbyregion)

                  region  total_students  percentage_of_students
0               Scotland            3446               10.572822
1    East Anglian Region            3340               10.247599
2          London Region            3216                9.867149
3           South Region            3092                9.486700
4   North Western Region            2906                8.916025
5   West Midlands Region            2582                7.921946
6      South West Region            2436                7.473997
7   East Midlands Region            2365                7.256159
8      South East Region            2111                6.476851
9                  Wales            2086                6.400147
10      Yorkshire Region            2006                6.154696
11          North Region            1823                5.593226
12               Ireland            1184                3.632682


In [14]:
query = '''

SELECT
    AVG(avg_clicks_per_student) AS overall_total_avg_clicks
FROM (
    SELECT
        id_student,
        AVG(sum_click) AS avg_clicks_per_student
    FROM
        studentVle
    GROUP BY
        id_student
) AS student_avg_clicks;

'''
result_student_interaction = pd.read_sql_query(query, conn)
print(result_student_interaction)

   overall_total_avg_clicks
0                  3.277612


In [15]:
query_create_fact_table = '''
CREATE TABLE fact_table AS
SELECT
    si.code_module,
    (COUNT(DISTINCT CASE WHEN si.final_result IN ('Pass', 'Distinction') THEN si.id_student END) * 100.0 / COUNT(DISTINCT si.id_student)) AS completion_rate,
    (COUNT(DISTINCT CASE WHEN si.final_result = 'Distinction' THEN si.id_student END) * 100.0 / COUNT(DISTINCT si.id_student)) AS distinct_completion_rate,
    AVG(sas.score) AS average_score,
    AVG(DISTINCT sv.date) AS average_time_spent,
    COUNT(DISTINCT sv.id_student) AS unique_students,
    (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM studentInfo)) AS percentage_of_students
FROM
    studentInfo si
JOIN
    studentAssesstment sas ON si.id_student = sas.id_student
JOIN
    assesstments a ON sas.id_assessment = a.id_assessment
JOIN
    studentVle sv ON si.id_student = sv.id_student
GROUP BY
    si.code_module;
'''

# Execute the query
conn.execute(query_create_fact_table)

# Commit the changes
conn.commit()

In [20]:
query = '''

SELECT
   *
FROM
    fact_table
LIMIT 6;


'''
result_fact = pd.read_sql_query(query, conn)
print(result_fact)

  code_module  completion_rate  distinct_completion_rate  average_score  \
0         AAA        78.286558                  6.499261      73.397366   
1         BBB        62.995299                 11.366689      79.195723   
2         CCC        45.620239                 13.547334      78.893889   
3         DDD        54.982094                  8.068254      74.228109   
4         EEE        71.090047                 15.338216      78.626570   
5         FFF        59.901478                 11.001642      79.615913   

   average_time_spent  unique_students  percentage_of_students  
0          122.500000              677             5909.081705  
1          122.469178             5956            43535.114902  
2          122.000000             3676            86483.794066  
3          122.000000             4747            75189.246157  
4          123.975945             2321            26543.260823  
5          122.000000             6090           157404.102108  
